In [59]:
# Load the annotation file
anno_path = '/workspace/work/O2ONet/data/annotations_minus_unavailable_yt_vids.pkl'

import pickle as pkl

f = open(anno_path, 'rb')
anno = pkl.load(f)
f.close()

In [60]:
# For i3d based bbox features

import numpy as np
import torch
import torchvision
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection import FasterRCNN
from torch import nn
import torchvision.transforms as transforms
import pytorchvideo.models as models
import torch.nn.functional as F
from collections import OrderedDict
from torch import nn

class FeatureExtractor(nn.Module):

    def __init__(self, submodule, layer):
        super(FeatureExtractor, self).__init__()
        self.pretrain_model = submodule
        self.layer = layer
        
        self.layer_list = list(self.pretrain_model._modules['blocks']._modules.keys())
        output_layer = self.layer_list[self.layer]  # just change the number of the layer to get the output

        self.children_list = []
        for (name, comp_layer) in self.pretrain_model._modules['blocks'].named_children():
            self.children_list.append(comp_layer)
            if name == output_layer:
                break
        self.feature_extrac_net = nn.Sequential(*self.children_list)
        self.pretrain_model = None

    def forward(self, image):
        feature = self.feature_extrac_net(image)
        return feature

from pytorchvideo.data.encoded_video import EncodedVideo
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import  NormalizeVideo
from pytorchvideo import transforms
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

import numpy as np

def read_gif(gif_path):
    """read gif and return dictionary as key ['video'] and value the tensor of size(CxTxHXW)"""

    video = EncodedVideo.from_path(gif_path)
    video = video.get_clip(0, 5) # get_clip fetches the clip from starting time to ending time

    return video

In [70]:
# Master Feature Generator for VSGNet
import torch

'''
get the current dictionary - DONE
get the current gif_folder - DONE
get the i3d feature extractor
and the transform
and the device
'''

def master_feature_generator(current_file, gif_folder, i3d_feature_extractor, i3d_transform, device):

    current_dict = torch.load(current_file)
    # temp_shape = current_dict['i3d_fmap'].shape
    
    # if int(temp_shape[-1]) == 80 and int(temp_shape[-2]) == 45:
    #     return current_dict, -1
    # if 'i3d_fmap' in list(current_dict.keys()):
        # return current_dict, -1
    # Getting details to load the GIF
    yt_id = current_dict['metadata']['yt_id']
    frame_index = current_dict['metadata']['frame no.']

    window_size = 5

    # Loading the gif    
    filename = yt_id + '_' + str(frame_index) + '_' + str(window_size) + '.gif'
    import os
    file_location = os.path.join(gif_folder, filename)

    # i3d features
    temp_i3d_video = read_gif(file_location)
    temp_i3d_video = i3d_transform(temp_i3d_video)["video"]
    temp_i3d_video = temp_i3d_video.unsqueeze(0).to(device)

    temp_i3d_feature_map = i3d_feature_extractor(temp_i3d_video)
    current_dict['i3d_fmap'] = torch.mean(temp_i3d_feature_map[0], dim=1).to(torch.device('cpu'))
    
    current_dict['bboxes'] = current_dict['bboxes'].to(torch.device('cpu'))
    current_dict['lr'] = current_dict['lr'].to(torch.device('cpu'))
    current_dict['mr'] = current_dict['mr'].to(torch.device('cpu'))
    current_dict['cr'] = current_dict['cr'].to(torch.device('cpu'))
    current_dict['frame_deep_features'] = current_dict['frame_deep_features'].to(torch.device('cpu'))
    
    return current_dict, 1

In [71]:

# Loading files required to generate the trackers
# Load the annotation file
anno_path = '/workspace/work/O2ONet/data/annotations_minus_unavailable_yt_vids.pkl'

import pickle as pkl

f = open(anno_path, 'rb')
annotations = pkl.load(f)
f.close()

gif_folder = '/workspace/data/data_folder/o2o/gifs_11'

# 2d cnn feature extractor
import torchvision
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
layer_no = 4

# i3d feature extractor
import pytorchvideo.models as models
model_name = "i3d_r50"
model = torch.hub.load("facebookresearch/pytorchvideo:main", model=model_name, pretrained=True)
model = model.to(device)
i3d_feature_net = FeatureExtractor(model, 4)

# i3d transform
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

from torchvision.transforms import Resize

i3d_transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(11),
            Resize((720,1280)),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std)    
        ]
    ),
)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [72]:
gif_folder = '/workspace/data/data_folder/o2o/gifs_11'
from glob import glob as glob
file_list = glob('/workspace/data/data_folder/o2o/gifs_11_features_vsgnet/*.pt')

from tqdm import tqdm as tqdm

issues = []

for f in tqdm(file_list):
    if 'DyNp5FJEheg_478_5' not in f:
        continue
    print('here')
    res, success = master_feature_generator(f, gif_folder, i3d_feature_net, i3d_transform, device)

    try:
        res, success = master_feature_generator(f, gif_folder, i3d_feature_net, i3d_transform, device)
        print(success)
        if success:
            torch.save(res, f)
    except:
        print("here")
        issues.append(f)

  0%|          | 0/2055 [00:00<?, ?it/s]

here


  0%|          | 1/2055 [00:00<18:17,  1.87it/s]


RuntimeError: Failed to open video DyNp5FJEheg_478_5.gif. [Errno 1094995529] Invalid data found when processing input: '<none>'

In [73]:
from glob import glob as glob
file_list = glob('/workspace/data/data_folder/o2o/gifs_11_features_vsgnet/*.pt')

issue_files = []
issue_indices = []

from tqdm import tqdm as tqdm

for i in tqdm(range(len(file_list))):
    
    data = torch.load( file_list[i] )
    
    if 'i3d_fmap' not in list(data.keys()):
        issue_files.append(f)
        issue_indices.append(i)
    

  2%|▏         | 51/2055 [00:10<07:09,  4.66it/s]


KeyboardInterrupt: 

In [56]:
issues

['/workspace/data/data_folder/o2o/gifs_11_features_vsgnet/DyNp5FJEheg_478_5.pt',
 '/workspace/data/data_folder/o2o/gifs_11_features_vsgnet/LcmEf0Aramg_886_5.pt']

In [64]:
data = torch.load('/workspace/data/data_folder/o2o/gifs_11_features_vsgnet/DyNp5FJEheg_478_5.pt')

In [65]:
data.keys()

dict_keys(['metadata', 'num_obj', 'bboxes', 'lr', 'mr', 'cr', 'object_pairs', 'num_relation', 'frame_deep_features'])